In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append("../")

### Import Desired Tires

In [3]:
all_tires = {
    "Hoosier_16x7.5-10_R20_7_cornering":{"long":None, "lat":None},
    "Hoosier_18x6.0-10_R20_7_cornering":{"long":None, "lat":None},
}

### Filter Data

In [4]:
camber = 0 # default camber
pressure = 12 * 6.89476 # default pressure
velocity = 25 * 1.60934 # default velocity
slip_angle = 0
tire_scaling = 2/3

tire_dict = all_tires.items()

for name, tire in tire_dict:
    try:
        df = pd.read_csv(f"../../processing/results/{name}.csv")
        tire["long"] = df[(df["pressure"] == pressure) & (df["velocity"] == velocity) & (df["camber"] == camber) & (df["slip"] == slip_angle)]
    except:
        print("Error getting long data for {0}".format(name))

    try:
        df = pd.read_csv(f"../../processing/results/{name}.csv")
        tire["lat"] = df[((df["velocity"] == velocity) & (df["pressure"] == pressure))]
    except:
        print("Error getting lateral data for {0}".format(name))

### Calculate Peak FY & MZ and Isolate Respective SAs

In [5]:
tires = []
SA_peak_FY = []
SA_peak_MZ = []
FZ_lst = []

normal_loads = sorted(np.array(tire["lat"]["load"].unique()))

for name, tire in all_tires.items():
    current_SA_peak_FY = []
    current_SA_peak_MZ = []
    current_FZ = []
    for load in normal_loads:
        if load > -300:
            continue
        
        temp = tire["lat"][tire["lat"]["load"] == load]
        temp2 = tire["lat"][tire["lat"]["load"] == load][(tire["lat"]["SA"] < 7.5) & (tire["lat"]["SA"] > -7.5)]

        FY_max_min = [min(temp["FY"]), max(temp["FY"])]
        abs_max_FY = max([abs(x) for x in FY_max_min])
        if abs_max_FY not in FY_max_min:
            abs_max_FY *= -1

        MZ_max_min = [max(temp2["MZ"])]
        abs_max_MZ = max([abs(x) for x in MZ_max_min])
        if abs_max_MZ not in MZ_max_min:
            abs_max_MZ *= -1
        
        peak_FY_SA = abs(tire["lat"][tire["lat"]["FY"] == abs_max_FY]["SA"].tolist()[0])
        peak_MZ_SA = abs(tire["lat"][tire["lat"]["MZ"] == abs_max_MZ]["SA"].tolist()[0])

        current_SA_peak_FY.append(peak_FY_SA)
        current_SA_peak_MZ.append(peak_MZ_SA)
        current_FZ.append(load)

        # print(peak_FY_SA, peak_MZ_SA)

        # if peak_MZ_SA < 3:
        #     print(peak_MZ_SA)
        #     plt.plot(temp["SA"], temp["FY"])
        #     plt.plot(temp["SA"], 100 * temp["MZ"])

        # SA_vals.append(peak_FY_SA)
        # FZ_current.append(load)

        # print(peak_FY_SA)

        # plt.title(f"{load}")
        # plt.plot(temp["SA"], temp["FY"])
    
    SA_peak_FY.append(current_SA_peak_FY)
    SA_peak_MZ.append(current_SA_peak_MZ)
    FZ_lst.append(current_FZ)
    tires.append(name[:23])

C:\Users\rober\AppData\Local\Temp\ipykernel_12364\2965598909.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp2 = tire["lat"][tire["lat"]["load"] == load][(tire["lat"]["SA"] < 7.5) & (tire["lat"]["SA"] > -7.5)]


### Save SA to .csv

In [6]:
diff_arr = np.array(SA_peak_FY) - np.array(SA_peak_MZ)

output_df = pd.DataFrame()
output_df["Tire"] = [tire[8:] for tire in tires]
output_df["SA at Peak FY"] = SA_peak_FY
output_df["SA at Peak MZ"] = SA_peak_MZ
output_df["Average Difference"] = [round(sum(arr) / len(arr), 3) for arr in diff_arr]

output_df = output_df.sort_values("Average Difference")

output_df.to_csv("16_inch_peak_FY_MZ_SA.csv")
